In [147]:
import os
import re
from typing import List
from pathlib import Path
from langchain.docstore.document import Document
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import MarkdownHeaderTextSplitter

In [174]:
def hierarchical_markdown_split(md_text: str, path_prefix: str = "") -> list[Document]:
    """마크다운 문서를 계층적으로 분할합니다."""
    splitter = MarkdownHeaderTextSplitter(headers_to_split_on=[
        ("#", "title"),
        ("##", "section"),
        ("###", "subsection"),
        ("####", "subsubsection"),
        ("#####", "subsubsubsection")
    ])
    docs = splitter.split_text(md_text)

    result_docs = []
    current_title = None
    chunk_idx = 0
    for doc in docs:
        metadata = doc.metadata
        if "title" in metadata:
            current_title = metadata["title"]

        if current_title:
            chunk_idx += 1
            full_title = "" + current_title
            if "section" in metadata:
                full_title += f" / {metadata['section']}"
            if "subsection" in metadata:
                full_title += f" / {metadata['subsection']}"
            if "subsubsection" in metadata:
                full_title += f" / {metadata['subsubsection']}"
            if "subsubsubsection" in metadata:
                full_title += f" / {metadata['subsubsubsection']}"

            content = f"Title: {full_title}\n{doc.page_content}"
            doc = Document(page_content=content, metadata={
                **doc.metadata,
                "type": "documentation",
                "source": "dev_center_guide_allmd.md",
                "chunk_idx": chunk_idx
            })

        result_docs.append(doc)

    return result_docs

def load_markdown_file(file_path: str) -> str:
    """마크다운 파일을 로드합니다."""
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

# 마크다운 파일 로드 및 분할
str_md_file = load_markdown_file("data/dev_center_guide_touched.md")
docs_markdown = hierarchical_markdown_split(str_md_file)

print(f"마크다운 문서 분할 완료: {len(docs_markdown)}개 청크")

마크다운 문서 분할 완료: 119개 청크


In [175]:
# docs_markdown에서 'launchPurchaseFlow()' 문자열을 포함하는 Documnet 를 출력

cnt = 0
for doc in docs_markdown:
    if 'PNS' in doc.page_content:
        # print(f"문서 제목: {doc.metadata.get('title', '제목 없음')}")
        print(f"--- doc_index: {cnt} ---")
        print(f"{doc.page_content}[EOD]")  # 처음 200자만 출력
        # print("-" * 40)  # 구분선 
        cnt += 1
        
print(f"'PNS' 문자열을 포함하는 문서 개수: {cnt}")
        

--- doc_index: 0 ---
Title: 원스토어 인앱 결제 연동 가이드 / 원스토어 인앱결제 API V7(SDK V21) 연동 안내 및 다운로드 / 07. PNS (Payment Notification Service) 이용하기 / 개요
원스토어는 개발자를 위해 두 가지 Payment Notification Service를 제공합니다.
* PNS는 Payment Notification Service의 약자입니다.
* PNS는 모바일의 네트워크 연결 불안정성을 보완하기 위해 개발사가 지정한 서버로 개별 사용자의 결제 상태(결제 완료, 결제 취소)를 메시지로 전송하는 기능입니다.
* 정확히는 개발사가 지정한 서버에서 원스토어가 정의한 규칙에 맞추어 API를 구현하면 해당 API를 원스토어의 결제 담당 서버에서 호출하는 형태입니다.
* Server to Server라고 할지라도 네트워크 문제로 메세지 전송 실패가 발생하기 때문에 200 OK로 응답을 인지하지 못할 경우 반복하여 메시지가 전송될 수 있습니다.
아래의 결제 트랜젝션 변화에 대하여 Notification 메시지가 전송됩니다.  
- 인앱상품 결제 또는 결제취소가 발생하면 원스토어가 개발사 서버로 알림을 전송하는 PNS(Payment Notification Service)  
- 구독 상태가 변경되면 개발사 서버로 알림을 전송하는 SNS (Subscription Notifacation Service)  
> Notification은 발송/수신 서버의 상태에 따라 지연 또는 유실될 수 있으므로, notification 수신을 기준으로 상품(서비스)을 제공하는 것은 권장하지 않습니다.
정상적인 결제 건인지 Server to Server로 확인하기를 원하신다면 PNS notification을 이용하는 대신, 관련 서버 API로 조회하는 것을 권장합니다.
원스토어는 검증 및 모니터링 목적으로 결제 테스트를 진행할 수 있으며, 해당 테스트 건들도 결제/결제취소 시 동일하게 notification이 발송됩니

In [176]:
# fixed_model_name = "deepseek-coder:6.7b"
fixed_model_name = "exaone3.5:latest"
# fixed_model_name = "mistral:latest"

def embed_and_save(docs: List[Document], output_path: str):
    """문서를 임베딩하고 FAISS 데이터베이스로 저장합니다."""
    # 임베딩 모델 초기화
    embedding_model = OllamaEmbeddings(model=fixed_model_name)
    
    # FAISS 데이터베이스 생성 및 저장
    db = FAISS.from_documents(docs, embedding_model)
    db.save_local(output_path)
    print(f"✅ 임베딩 저장 완료: {output_path}")

# 모든 문서 통합
total_docs = docs_markdown #+ docs_kotlin
print(f"총 {len(total_docs)}개의 문서를 생성하였습니다.")
print(f"- 마크다운 문서: {len(docs_markdown)}개")
#print(f"- Kotlin 코드: {len(docs_kotlin)}개")

# 임베딩 생성 및 저장
output_dir = "models/faiss_vs_rag_iap_v2_11_" + fixed_model_name[:3]
os.makedirs(output_dir, exist_ok=True)
embed_and_save(total_docs, output_dir)

총 119개의 문서를 생성하였습니다.
- 마크다운 문서: 119개
✅ 임베딩 저장 완료: models/faiss_vs_rag_iap_v2_11_exa


In [182]:
###### Retriever Check from FAISS Vector DB ######

from langchain.embeddings import OllamaEmbeddings
from langchain.vectorstores import FAISS
from langchain_core.documents import Document
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate

# ✅ 3. 임베딩 모델 초기화 (Ollama)
embedding_model = OllamaEmbeddings(model=fixed_model_name)

# 저장된 데이터를 로드
loaded_db = FAISS.load_local(
    folder_path=output_dir,
    # index_name="index",
    embeddings=embedding_model,
    allow_dangerous_deserialization=True,
)

retriever = loaded_db.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 10, "fetch_k": 30, "lambda_mult": 0.7}
    # search_type="similarity",
    # search_kwargs={"k": 50}
)

res = retriever.invoke(
    "PNS(Payment Notification Service)란?"
)

print(f"검색된 문서 수: {len(res)}")

idx = 0
for doc in res: 
    print(f"--- doc_index: {idx} ---")
    print(doc.page_content)  # Print first 100 characters of each document
    # print(doc.metadata)
    # print('-' * 40)
    idx += 1

검색된 문서 수: 10
--- doc_index: 0 ---
Title: 원스토어 인앱 결제 연동 가이드 / 원스토어 인앱결제 API V7(SDK V21) 연동 안내 및 다운로드 / 06. 원스토어 인앱결제 서버 API (API V7) / 개요
원스토어 인앱결제 서버 API란 원스토어에서 결제된 인앱 상품의 데이터를 조회하거나 결제 상태를 변경하기 위한 Open API를 말합니다.  
해당 API를 사용하기 위해서는 OAuth 인증이 필요합니다.
--- doc_index: 1 ---
Title: 원스토어 인앱 결제 연동 가이드 / 원스토어 인앱결제 API V7(SDK V21) 연동 안내 및 다운로드 / 08. 정기 결제 적용하기 / IMMEDIATE_WITH_TIME_PRORATION
정기 결제 상품 A가 즉시 종료됩니다. 사용자는 한 달(4월 1일~30일) 요금을 결제 했는데 정기 결제 기간 중간에 상품을 변경 했으므로 월간 정기 결제 요금의 절반(1,000원)은 새 정기 결제에 적용됩니다. 그러나 새 정기 결제 요금은 연간 36,000원이므로 1,000원의 잔액은 10일(4월 16일~25일)에 해당합니다. 따라서 4월 26일에 새 정기 결제의 요금으로 36,000원이 청구되며, 다음 해부터 매년 4월 26일에 36,000원이 청구됩니다.
--- doc_index: 2 ---
Title: 원스토어 인앱 결제 연동 가이드 / 원스토어 인앱결제 API V7(SDK V21) 연동 안내 및 다운로드 / 07. PNS (Payment Notification Service) 이용하기 / PNS Notification 전송 정책 / PNS 전송 Example
1. 0회차(최초) 매세지는 지연 0초에 2020-05-17 13:10:00에 전송
2. 1회차 메시지는 지연 30초에 2020-05-17 13:10:30에 전송
3. 2회차 메시지는 지연 120초에 2020-05-17 13:12:30에 전송
4. 3회차 메시지는 지연 270초에 2020-05-17 13:17:00에

In [183]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


# RAG 체인 구성 요소
# embedding_model = OllamaEmbeddings(model=fixed_model_name)
llm = ChatOllama(model=fixed_model_name, temperature=0.3)

# 검색기 설정
retriever = loaded_db.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 20, "fetch_k": 100, "lambda_mult": 0.6}
)

# 프롬프트 템플릿
prompt_template = """
당신은 원스토어 인앱결제 전문가입니다. 주어진 컨텍스트를 바탕으로 질문에 답변해주세요.

답변 시 다음 사항을 고려해주세요:
1. 한국어로 명확하고 이해하기 쉽게 답변하세요
2. 코드 예시가 있다면 포함해주세요
3. 단계별로 설명해주세요
4. 개발자 관점에서 실용적인 정보를 제공해주세요
5. 반드시 embedding된 문서(컨텍스트) 내용을 토대로 LLM 지식을 추가해주세요. 
6. 컨텍스트에 없는 내용은 "해당 정보를 찾을 수 없습니다"라고 답변하세요
7. 원스토어 이외의 정보는 답변하지 마세요.
8. url 등을 노출할때 onstore가 포함되지 않는 경우 노출되지 않아야 합니다.
9. 원스토어에서 사용하는 PNS는 Payment Notification Service의 약자입니다. Push Notification Service와는 무관합니다.

컨텍스트:
{context}

질문: {question}

답변:"""

prompt = PromptTemplate.from_template(prompt_template)

# RAG 체인 구성
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print("✅ RAG 체인이 생성되었습니다!")

✅ RAG 체인이 생성되었습니다!


In [184]:
# 테스트 질문들
test_questions = [
    # "원스토어 인앱결제 초기화는 어떻게 하나요?",
    # "정기결제 구현 방법을 알려주세요",
    # "PNS(Payment Notification Service) 설정 방법은?",
    # "결제 검증은 어떻게 하나요?",
    # "관리형 상품과 구독형 상품의 차이점은?",
    "원스토어의 PNS(Payment Notification Service)란 무엇이고 어떻게 구현하나요? 메시지 수신 서버의 관점에서 Java SpringFramework로 구현한 코드를 예제를 알려주세요"
]

print("🤖 RAG 체인 테스트 시작\n")
print("=" * 80)

for i, question in enumerate(test_questions, 1):
    print(f"\n📝 질문 {i}: {question}")
    print("-" * 60)
    
    try:
        answer = rag_chain.invoke(question)
        print(answer)
    except Exception as e:
        print(f"❌ 오류 발생: {str(e)}")
    
    print("=" * 80)

🤖 RAG 체인 테스트 시작


📝 질문 1: 원스토어의 PNS(Payment Notification Service)란 무엇이고 어떻게 구현하나요? 메시지 수신 서버의 관점에서 Java SpringFramework로 구현한 코드를 예제를 알려주세요
------------------------------------------------------------
원스토어의 PNS (Payment Notification Service)는 앱 내에서 발생한 결제 이벤트를 개발자에게 실시간으로 알림을 제공하는 서비스입니다. 이 서비스는 앱의 구매 이벤트를 감지하고, 개발자가 지정한 서버로 해당 이벤트 정보를 포함한 메시지를 전송합니다. 이를 통해 개발자는 서버 측에서 구매 이벤트를 처리하고 필요한 동작을 수행할 수 있습니다.

### PNS 구현 방법

PNS 메시지는 일반적으로 JSON 형식으로 전송되며, 메시지에는 구매 정보, 앱 정보 등이 포함됩니다. 메시지 수신 서버의 관점에서 Java Spring Framework를 사용하여 간단한 구현 예제를 살펴보겠습니다.

#### 1. **프로젝트 설정**
   - Maven 또는 Gradle을 사용하여 Spring Boot 프로젝트를 설정합니다.
   - 필요한 의존성을 추가합니다 (예: Spring Web, Spring Boot Starter).

#### 2. **PNS 메시지 수신 엔드포인트 설정**
   - `@RestController` 어노테이션을 사용하여 메시지 수신 엔드포인트를 정의합니다.
   - `@RequestBody`와 `@ResponseBody` 어노테이션을 사용하여 메시지를 파싱하고 응답을 생성합니다.

#### 3. **Java 코드 예제**

아래는 간단한 예제 코드입니다. 이 코드는 HTTP POST 요청을 통해 PNS 메시지를 수신하고 처리하는 방법을 보여줍니다.

```java
import org.springframework.boot.web.servlet.framework.ServletI